## Get starred songs

In [1]:
# import required modules
import sys
import spotipy
import spotipy.util as util
import pandas as pd

In [3]:
df = pd.read_pickle('data/starred_songs.pkl')

In [2]:
# spotify api parameters
username = 'colinspear1'
client_id='ce2cc5f48d51431d80ff674bfba165e5'
client_secret='e36aec36c3174decbce246226e6d8b7a'
redirect_uri = 'http://localhost/'
scope = 'user-library-read'


# get token
token = util.prompt_for_user_token(
    username=username,
    scope=scope, 
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri
)

# instantiate spotipy
sp = spotipy.Spotify(auth=token)

In [2]:
def saved_tracks(session, limit):
    # collect saved tracks
    # improve by not limiting result to 350 - extend indefinitely
    song_list = []
    i = 0
    while i < 100:
        results = session.current_user_saved_tracks(limit=50, offset=i)
        for item in results['items']:
            song_list.append(item)
        i += 50

## Save song features and analysis

In [9]:
def track_info_df(song_list):
    """
    Takes a list of dictionaries returned from spotipy json 
    responses and creates a pandas data frame of its 
    characteristics
    
    :song_list: list of dictionaries of track characteristics
    """
    # initiate song dictionary
    song_dict = dict(
        time_added=[],
        release_date=[],
        release_date_precision=[],
        artist_name=[],
        artist_id=[],
        song_id=[],
        song_length_ms=[],
        song_name=[],
        popularity=[], 
        loudness = [], 
        tempo = [], 
        tempo_confidence = [], 
        time_signature = [],
        time_sig_conf = [], 
        key = [], 
        key_confidence = [], 
        mode = [], 
        mode_confidence = [], 
        danceability = [],
        energy = [], 
        speechiness = [], 
        acousticness = [], 
        instrumentalness = [], 
        liveness = [], 
        valence = [], 
    )

    # collect saved track data
    for i, song in enumerate(song_list):
        if song_list[i]['track']['id'] in song_dict['song_id']:
            continue
        else:        
            song_dict['time_added'].append(song_list[i]['added_at'])
            song_dict['release_date'].append(song_list[i]['track']['album']['release_date'])
            song_dict['release_date_precision'].append(song_list[i]['track']['album']['release_date_precision'])
            song_dict['artist_name'].append(song_list[i]['track']['album']['artists'][0]['name'])
            song_dict['artist_id'].append(song_list[i]['track']['artists'][0]['id'])
            song_dict['song_id'].append(song_list[i]['track']['id'])
            song_dict['song_length_ms'].append(song_list[i]['track']['duration_ms'])
            song_dict['song_name'].append(song_list[i]['track']['name'])
            song_dict['popularity'].append(song_list[i]['track']['popularity'])

            audio_analysis = sp.audio_analysis(song_list[i]['track']['id'])
            song_dict['loudness'].append(audio_analysis['track']['loudness'])
            song_dict['tempo'].append(audio_analysis['track']['tempo'])
            song_dict['tempo_confidence'].append(audio_analysis['track']['tempo_confidence'])
            song_dict['time_signature'].append(audio_analysis['track']['time_signature'])
            song_dict['time_sig_conf'].append(audio_analysis['track']['time_signature_confidence'])
            song_dict['key'].append(audio_analysis['track']['key'])
            song_dict['key_confidence'].append(audio_analysis['track']['key_confidence'])
            song_dict['mode'].append(audio_analysis['track']['mode'])
            song_dict['mode_confidence'].append(audio_analysis['track']['mode_confidence'])

            audio_features = sp.audio_features(song_list[i]['track']['id'])
            song_dict['danceability'].append(audio_features[0]['danceability'])
            song_dict['energy'].append(audio_features[0]['energy'])
            song_dict['speechiness'].append(audio_features[0]['speechiness'])
            song_dict['acousticness'].append(audio_features[0]['acousticness'])
            song_dict['instrumentalness'].append(audio_features[0]['instrumentalness'])
            song_dict['liveness'].append(audio_features[0]['liveness'])
            song_dict['valence'].append(audio_features[0]['valence'])

    fave_songs = pd.DataFrame(song_dict)
    return fave_songs
    #fave_songs.to_pickle('starred_songs.pkl')

## Get Discover Weekly song data

In [3]:
def discover_weekly_track_list(session):
    """Returns list of Discover Weekly song
    characteristics
    
    :session: authorized spotipy instance
    """
    offset = 0
    playlists = []
    dw_tracks = []
    while offset < 100:
        playlists.append(sp.user_playlists(username, offset=offset))
        offset += 50

    for i in playlists:
        for pl in i['items']:
            name = pl['name']
            if name == 'Discover Weekly':
                playlist_id = pl['id']
                dw_tracks = sp.user_playlist_tracks(username, playlist_id=playlist_id)
            else:
                continue
        
    return dw_tracks['items']

In [4]:
dw_tracks = discover_weekly_track_list(sp)

In [10]:
dw_tracks_df = track_info_df(dw_tracks)

Grabbing data from song number 0
Grabbing data from song number 25


In [11]:
dw_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 25 columns):
time_added                30 non-null object
release_date              30 non-null object
release_date_precision    30 non-null object
artist_name               30 non-null object
artist_id                 30 non-null object
song_id                   30 non-null object
song_length_ms            30 non-null int64
song_name                 30 non-null object
popularity                30 non-null int64
loudness                  30 non-null float64
tempo                     30 non-null float64
tempo_confidence          30 non-null float64
time_signature            30 non-null int64
time_sig_conf             30 non-null float64
key                       30 non-null int64
key_confidence            30 non-null float64
mode                      30 non-null int64
mode_confidence           30 non-null float64
danceability              30 non-null float64
energy                    30 non-null fl

In [6]:
dw_tracks_df.head()

,time_added,release_date,release_date_precision,artist_name,artist_id,song_id,song_length_ms,song_name,popularity,loudness,...,key_confidence,mode,mode_confidence,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
0,2019-08-05T02:30:00Z,2019-07-12,day,Joanna Sternberg,2qDa6aEDJtMIQMb6IddWaI,50bMSujg47Xc9LRuy6Xra9,127210,For You,26,-20.565,...,0.724,1,0.715,0.634,0.0636,0.0593,0.966000,0.0002,0.106,0.787
1,2019-08-05T02:30:00Z,2019-07-26,day,The Birthmarks,6oVUKPnleGC7kPhbXZJELE,72x5c7MQl5hZcv94a5noiv,174546,Fan Fiction,31,-6.018,...,0.096,1,0.493,0.411,0.7600,0.0312,0.000006,0.6650,0.147,0.592
2,2019-08-05T02:30:00Z,2019-06-14,day,The Family Daptone,034zK86rrt3F3P6ClQuM7p,5AYPDmFmyBetpIB370O4V9,232071,Hey Brother (Do Unto Others),31,-13.217,...,0.682,1,0.441,0.806,0.5050,0.0516,0.846000,0.1370,0.303,0.930
3,2019-08-05T02:30:00Z,2003,year,Lightnin' Hopkins,6EZzVXM2uDRPmnHWq9yPDE,4eodwADKbQcexyMRDdRYGY,155733,Bad Luck And Trouble (49-50),33,-10.421,...,0.600,1,0.529,0.457,0.2870,0.0349,0.982000,0.1120,0.104,0.246
4,2019-08-05T02:30:00Z,2013-09-17,day,Bill Callahan,7gqsi6aBSkRMJoL9psKqMr,0taVevkjpqGtGnVnIIzaJM,236600,Small Plane,32,-17.968,...,0.687,1,0.660,0.570,0.1630,0.0416,0.899000,0.6170,0.104,0.101
